In [6]:
%%file server_profiler.py
# -*- coding: utf8 -*-
import json
import os
import traceback
from flask import Flask, Response, jsonify, render_template, request,redirect,url_for,session
from model_loader import ModelLoader
import requests
import AIFlyserving
import yaml
import functools
import re
import time

from pprint import pprint as pp
from . import storage
import logging

#from redis_test import RedisDict

ai_server_config = {}

if 'AI_CONFIG' in os.environ:
    if os.path.exists(os.environ['AI_CONFIG']):
        with open(os.environ['AI_CONFIG']) as f:
            ai_server_config = yaml.load(f)

ROTATION_FILE_PATH = ai_server_config["rotation_status_file"]
app = Flask(__name__)
app.logger_name = "AI.app"
models_loaded = {}
model_loader = ModelLoader(ai_server_config)


CONF = {}
collection = None


logger = logging.getLogger("AI-dashboard")

_is_initialized = lambda: True if CONF else False


try:
    if 'MODELS_TO_LOAD' in os.environ:
        models_to_load=os.environ['MODELS_TO_LOAD']
        model_list=[]
        model_version_list_args=models_to_load
        model_version_list=model_version_list_args.split(',')
        for model_version in model_version_list:
            model_name=model_version.split(':')[0]
            version=model_version.split(':')[1]
            model_list.append([model_name,version])
        models_to_load =model_list# json.loads(os.environ['MODELS_TO_LOAD'])
        print(models_to_load)
        models_loaded = model_loader.get_models_from_list(models_to_load)

except requests.exceptions.HTTPError as e:
    app.logger.error("Meta Service has thrown %s, the error is %s and stack trace is %s"
                     %(e.response.status_code, e.message, str(traceback.format_exc())))
    raise RuntimeError("Meta Service has thrown '{}' , the error is {} and stack trace is {}".format(e.response.status_code, e.message, str(traceback.format_exc())))

app.logger.info("Loaded models are: " + json.dumps(models_loaded.keys()))
from abc import ABCMeta, abstractmethod

import json
import logging
from jinja2 import Template


def gen_python_api(json_data, model_name,model_version,endpoint="http://127.0.0.1:8400"):
    """
    Generate TensorFlow SDK in Python.
    Args:
    generated_tensor_data: Example is {"keys": [[1.0], [2.0]], "features": [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}
    """

    code_template = """#!/usr/bin/env python

    import requests

    def main():
        #endpoint = "http://127.0.0.1:8000"
        endpoint = {{endpoint}}
        param={"model_name": "{{ model_name }}", "model_version": "{{ model_version }}"}
        json_data = {{json_data}}
        result = requests.post(endpoint, param=param,json=json_data)
        print(result.text)

    if __name__ == "__main__":
        main()
    """

    generated_tensor_data_string = json.dumps(json_data)
    template = Template(code_template)
    generate_code = template.render(
        model_name=model_name, model_version=model_version,json_data=generated_tensor_data_string,endpoint=endpoint)
    logging.debug("Generate the code in Python:\n{}".format(generate_code))
    return generate_code





class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
import requests
def predict_client(model_server_url,params,data):
    model_server_url =model_server_url+'/predict'# "http://localhost:8000/predict"
    #params = {"model_id":"fib_model", "model_version":"1.0.2"}
    #data = json.dumps({"data":11})
    response = requests.post(model_server_url, params = params, json = data)
    return response.content#eval(response.content)['result']


app.secret_key='\xf1\x92Y\xdf\x8ejY\x04\x96\xb4V\x88\xfb\xfc\xb5\x18F\xa3\xee\xb9\xb9t\x01\xf0\x96' 
@app.route("/login",methods=['POST','GET'])
def login():
    error = None
    if request.method == 'POST':
        session['username']=request.form['username']
        session['password']=request.form['password']
        if request.form['username'] != 'admin' or request.form['password'] != 'admin123': 
                error= "sorry"
        else:
            return redirect(url_for('index'))
    return render_template('login.html',error=error)
#test=RedisDict(namespace='web_ui')
class InferenceService(object):
    """
    The abstract class for inference service which should implement the method.
    """
    __metaclass__ = ABCMeta

    def __init__(self,model_name,model_version, model_graph_signature=None, platform=None):
        self.model_name = model_name
        self.model_base_path = None
        self.model_version = model_version
        self.model_graph_signature = model_graph_signature
        self.platform = platform
model_name_service_map={}
for m_list in models_to_load:
    model_name,model_version=m_list
    generated_tensor_data={"keys": [[1.0], [2.0]], "features": [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}
    graph_api_info=gen_python_api(generated_tensor_data,model_name,model_version=model_version)
    model_name_service_map[model_name]=InferenceService(model_name=model_name,
                                                        model_version=model_version,
                                                        model_graph_signature=graph_api_info,
                                                        platform=model_name.split('_')[0])
@app.route("/index", methods=["GET"])
def index():
    if session.get('username')=='admin' and session.get('password')=='admin123':
        """
        model_name_service_map_redis={}
        dict_redis=test['model_info']
        for kk in dict_redis['model_config_list']:
            model_name_service_map_redis[kk['name']]=InferenceService(kk['name'],kk['base_path'],kk['platform'])
        print(model_name_service_map_redis)        
        """

        return render_template("index.html", model_name_service_map=model_name_service_map)
    return "you are not logged in" 
@app.route('/json_inference', methods=["GET"])
def json_inference():
    return render_template('json_inference.html')

from gen_client import python_client

@app.route('/run_json_inference', methods=['POST'])
def run_json_inference():
    json_data_string = request.form["json_data"]
    json_data = json.loads(json_data_string)
    model_name = request.form["model_name"]
    model_name_re=model_name.split(':')[0]
    model_name_version=model_name.split(':')[1]
    param={"model_id":model_name_re,"model_version":model_name_version}
    request_json_data = {"model_name": model_name, "data": json_data}
    #predict_client
    

    predict_result = predict_client("http://0.0.0.0:8000",param, json.dumps(json_data))
    print(param,json_data,predict_result)  
    return render_template('json_inference.html', predict_result=predict_result)


@app.route('/elb-healthcheck', methods=['GET'])
def elb_healthcheck():
    try:
        if os.path.isfile(ROTATION_FILE_PATH):
            with open(ROTATION_FILE_PATH) as fd:
                lines = (fd.read()).strip()
                if lines == '1':
                    #TODO: uptime, requests and capacity have to be computed
                    result = {"uptime": 0, "requests": 0, "capacity": 100}
                    return jsonify(result)
        response = jsonify({'Message': "Out of rotation"})
        response.status_code = 500
        return response
    except Exception as e:
        response = jsonify({'Message': "Out of rotation"})
        response.status_code = 500
        return response

@app.route('/rotation_status', methods=['POST'])
def rotation_status():
    try:
        state = request.args.get('state')
        if state is not None:
            if state == "oor" or state == "OOR":
                write_rotation_status(ROTATION_FILE_PATH, '0')
                result = {'Message': "Taking out of rotation"}
                return jsonify(result)
            elif state == "bir" or state == "BIR":
                write_rotation_status(ROTATION_FILE_PATH, '1')
                result = {'Message': "Taking back in rotation"}
                return jsonify(result)
            else:
                response = jsonify({'Message': "Bad Request"})
                response.status_code = 400
                return response
        else:
            response = jsonify({'Message': "Bad Request"})
            response.status_code = 400
            return response
    except Exception as e:
        result = {'Message': str(e)}
        return jsonify(result)


@app.route('/models-loaded', methods=['GET'])
def models_available():
    response = jsonify({"result":models_loaded.keys()})
    response.status_code = 200
    return response

@app.route('/health', methods=['GET'])
def health():
    app.logger.debug("Health Check")
    try:
        if os.path.isfile(ROTATION_FILE_PATH):
            with open(ROTATION_FILE_PATH) as fd:
                lines = (fd.read()).strip()
                if lines == '1':
                    result = {"version": AIFlyserving.__version__, "health_status": "OK"}
                    return json.dumps(result)
        response = jsonify({'Message': "Out of rotation"})
        response.status_code = 500
        return response
    except Exception as e:
        exc_traceback = str(traceback.format_exc())
        app.logger.error("Exception occurred: " + str(e.message) + "," + exc_traceback)
        response = jsonify({"stack_trace": exc_traceback})
        response.status_code = 500
        return response
@app.route('/API_stat', methods=["GET"])
def API_stat():
    return render_template('API_stat.html')

@app.route('/image_inference', methods=["GET"])
def image_inference():
    return render_template('image_inference.html')



@app.route('/run_image_inference', methods=['POST'])
def run_image_inference():
  file = request.files['image']
  file_path = os.path.join(application.config['UPLOAD_FOLDER'], file.filename)
  file.save(file_path)

  channel_layout = "RGB"
  if "channel_layout" in request.form:
    channel_layout_ = request.form["channel_layout"]
    if channel_layout_ in ["RGB", "RGBA"]:
        channel_layout = channel_layout_

  run_profile = ""
  if "run_profile" in request.form:
    run_profile = request.form["run_profile"]

  image_file_path = os.path.join(application.config['UPLOAD_FOLDER'],
                                 file.filename)
  predict_result = python_predict_client.predict_image(
    image_file_path, channel_layout=channel_layout, run_profile=run_profile, port=args.port)

  return render_template(
      'image_inference.html',
      image_file_path=image_file_path,
      predict_result=predict_result)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        try:
            input = json.loads(request.data)
        except ValueError as e:
            stack_trace = traceback.format_exc()
            app.logger.error("Json Decoding failed. Check if the payload is correct. Payload is " +
                             str(request.data) + " and the stack_trace is " + stack_trace)
            response = jsonify({"result": "NA", "error": "Json Decoding failed. Check if the payload is correct",
                                "exception": str(e), "stack_trace": stack_trace})
            response.status_code = 400
            return response
        model_id = request.args.get('model_id')
        model_version = request.args.get('model_version')
        key = (model_id, model_version)

        try:
            curr_model = models_loaded[key]
        except KeyError:
            app.logger.error("Model: (%s,%s) doesn't exist " %(model_id, model_version))
            response = jsonify({"result":"NA", "stack_trace" : "Model: (%s,%s) doesn't exist. Deploy this model on AI server " %(model_id, model_version)})
            response.status_code = 400
            return response
        input2=eval(input)['data']
        print(input2,type(input2))
        output = curr_model.predict(input2)
        response = jsonify(json.loads(json.dumps(output,cls=NumpyEncoder)))
        #jsonify(json.loads(json.dumps({"result":output,"status":"success!"},cls=NumpyEncoder))))
        response.status_code = 200
        return response
    except Exception as e:
        exc_traceback = str(traceback.format_exc())
        app.logger.error("Exception occurred: " + str(e) + "," + exc_traceback)
        response = jsonify({"result":"NA", "stack_trace": exc_traceback})
        response.status_code = 500
        return response


def lock(lockfile):
    import fcntl
    lockfd = open(lockfile, 'w+')
    fcntl.flock(lockfd, fcntl.LOCK_EX | fcntl.LOCK_NB)
    return lockfd


def unlock(lockfd):
    import fcntl
    fcntl.flock(lockfd, fcntl.LOCK_UN)

def write_rotation_status(file_path, status):
    if os.path.isfile(file_path):
        with open(file_path) as f:
            lines = f.read().strip()
            if lines == status:
                return
    lockfile = file_path + '.lock'
    if not os.path.exists(lockfile):
        fd = open(lockfile, 'w+')
        fd.close()

    lockfd = lock(lockfile)
    file = open(file_path, 'w+')
    file.write(status)
    file.close()
    unlock(lockfd)
"""for dashboard"""
class Measurement(object):
    """represents an endpoint measurement"""
    DECIMAL_PLACES = 6

    def __init__(self, name, args, kwargs, method, context=None):
        super(Measurement, self).__init__()
        self.context = context
        self.name = name
        self.method = method
        self.args = args
        self.kwargs = kwargs
        self.startedAt = 0
        self.endedAt = 0
        self.elapsed = 0

    def __json__(self):
        return {
            "name": self.name,
            "args": self.args,
            "kwargs": self.kwargs,
            "method": self.method,
            "startedAt": self.startedAt,
            "endedAt": self.endedAt,
            "elapsed": self.elapsed,
            "context": self.context
        }

    def __str__(self):
        return str(self.__json__())

    def start(self):
        # we use default_timer to get the best clock available.
        # see: http://stackoverflow.com/a/25823885/672798
        self.startedAt = time.time()

    def stop(self):
        self.endedAt = time.time()
        self.elapsed = round(
            self.endedAt - self.startedAt, self.DECIMAL_PLACES)


def is_ignored(name, conf):
    ignore_patterns = conf.get("ignore", [])
    for pattern in ignore_patterns:
        if re.search(pattern, name):
            return True
    return False


def measure(f, name, method, context=None):
    logger.debug("{0} is being processed.".format(name))
    if is_ignored(name, CONF):
        logger.debug("{0} is ignored.".format(name))
        return f

    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        if 'sampling_function' in CONF and not callable(CONF['sampling_function']):
            raise Exception(
                "if sampling_function is provided to flask-profiler via config, "
                "it must be callable, refer to: "
                "https://github.com/muatik/flask-profiler#sampling")

        if 'sampling_function' in CONF and not CONF['sampling_function']():
            return f(*args, **kwargs)

        measurement = Measurement(name, args, kwargs, method, context)
        measurement.start()

        try:
            returnVal = f(*args, **kwargs)
        except:
            raise
        finally:
            measurement.stop()
            if CONF.get("verbose", False):
                pp(measurement.__json__())
            collection.insert(measurement.__json__())

        return returnVal

    return wrapper


def wrapHttpEndpoint(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        context = {
            "url": request.base_url,
            "args": dict(request.args.items()),
            "form": dict(request.form.items()),
            "body": request.data.decode("utf-8", "strict"),
            "headers": dict(request.headers.items()),
            "func": request.endpoint,
            "ip": request.remote_addr
        }
        endpoint_name = str(request.url_rule)
        wrapped = measure(f, endpoint_name, request.method, context)
        return wrapped(*args, **kwargs)

    return wrapper


def wrapAppEndpoints(app):
    """
    wraps all endpoints defined in the given flask app to measure how long time
    each endpoints takes while being executed. This wrapping process is
    supposed not to change endpoint behaviour.
    :param app: Flask application instance
    :return:
    """
    for endpoint, func in app.view_functions.items():
        app.view_functions[endpoint] = wrapHttpEndpoint(func)


def profile(*args, **kwargs):
    """
    http endpoint decorator
    """
    if _is_initialized():
        def wrapper(f):
            return wrapHttpEndpoint(f)

        return wrapper
    raise Exception(
        
        "before measuring anything, you need to call init_app()")
CONF={
    "storage": {
        "engine": "mongodb",
    }
}
#CONF.get("storage", {})
collection = storage.getCollection(CONF.get("storage", {}))

@app.route("/dashboard")
def dashboard():
    return render_template("dashboard.html")

@app.route("/api/measurements/")
def filterMeasurements():
    args = dict(request.args.items())
    measurements = collection.filter(args)
    return jsonify({"measurements": list(measurements)})

@app.route("/api/measurements/grouped")
def getMeasurementsSummary():
    args = dict(request.args.items())
    measurements = collection.getSummary(args)
    return jsonify({"measurements": list(measurements)})

@app.route("/api/measurements/<measurementId>")
def getContext(measurementId):
    return jsonify(collection.get(measurementId))

@app.route("/api/measurements/timeseries/")
def getRequestsTimeseries():
    args = dict(request.args.items())
    return jsonify({"series": collection.getTimeseries(args)})

@app.route("/api/measurements/methodDistribution/")
def getMethodDistribution():
    args = dict(request.args.items())
    return jsonify({
        "distribution": collection.getMethodDistribution(args)})

@app.route("/db/dumpDatabase")
def dumpDatabase():
    response = jsonify({
        "summary": collection.getSummary()})
    response.headers["Content-Disposition"] = "attachment; filename=dump.json"
    return response

@app.route("/db/deleteDatabase")
def deleteDatabase():
    response = jsonify({
        "status": collection.truncate()})
    return response

@app.after_request
def x_robots_tag_header(response):
    response.headers['X-Robots-Tag'] = 'noindex, nofollow'
    return response
class Profiler(object):
    """ Wrapper for extension. """

    def __init__(self, app=None):
        self._init_app = init_app
        if app is not None:
            self.init_app(app)

    def init_app(self, app):
        init = functools.partial(self._init_app, app)
        app.before_first_request(init)
def init_app(app):
    global collection, CONF

    collection = storage.getCollection(CONF.get("storage", {}))

    wrapAppEndpoints(app)
profiler = Profiler()  # You can have this in another module
profiler.init_app(app)

Overwriting server_profiler.py


In [2]:
"/api/measurements/grouped".format('dppp')

'/api/measurements/grouped'

In [3]:
CONF={
    "storage": {
        "engine": "mongodb",
    }
}
CONF.get("storage", {})

{'engine': 'mongodb'}

In [ ]:
{% include "header.html" %}
<br />
<head>
    <title></title>


    <link href="static/dist/css/main.css" rel="stylesheet">
    

</head>





<body>

<div class="container">
    
    
    <div class="col-md-10 col-md-offset-1">

    <div class="navbar-collapse collapse">
      <ul class="nav navbar-nav navbar-left" role="tablist">
        <li role="presentation">
            <a href="#tab-filtering" data-target="#tab-filtering" role="tab" data-toggle="tab" data-tab-history="true" data-tab-history-changer="push" data-tab-history-update-url="true">Filtering</a>
        </li>
          <li role="presentation">
            <a href="#tab-settings" data-target="#tab-settings" role="tab" data-toggle="tab" data-tab-history="true" data-tab-history-changer="push" data-tab-history-update-url="true">Settings</a>
        </li>
      </ul>
      <iframe src="https://ghbtns.com/github-btn.html?user=leepand&repo=AIFly&type=star&count=true" frameborder="0" scrolling="0" width="90px" height="20px" style='margin-top: 15px;' class="pull-right"></iframe>
    </div>
  </div>
    
    
    
    <div class="col-xs-12 col-md-10 col-md-offset-1">
        <div class="row">
            <div class="tab-content">

  

                <div role="tabpanel" class="tab-pane" id="tab-filtering">
                    <div class="panel panel-default">
                      <div class="panel-heading clearfix">
                        <h3 class="panel-title pull-left">Filtering</h3>
                        <button type="button" class="btn btn-default pull-right clear-filter" style="padding: 3px 7px; margin-bottom: 0; font-size: 13px;">Clear</button>
                      </div>
                      <div class="panel-body">
                          <div class="table-responsive">
                            <table class="table table-hover" id="filteredTable">
                                <thead>
                                    <tr class="filter-row">
                                        <th>
                                            <select class='method form-control'>
                                                <option value="ALL">ALL</option>
                                                <option value="GET">GET</option>
                                                <option value="POST">POST</option>
                                                <option value="PUT">PUT</option>
                                                <option value="DELETE">DELETE</option>
                                                <option value="PATCH">PATCH</option>
                                                <option value="COPY">COPY</option>
                                                <option value="HEAD">HEAD</option>
                                                <option value="OPTIONS">OPTIONS</option>
                                                <option value="LINK">LINK</option>
                                                <option value="UNLINK">UNLINK</option>
                                                <option value="PURGE">PURGE</option>
                                                <option value="LOCK">LOCK</option>
                                                <option value="UNLOCK">UNLOCK</option>
                                                <option value="PROPFIND">PROPFIND</option>
                                                <option value="VIEW">VIEW</option>
                                            </select>
                                        </th>
                                        <th><input type="text" class="filtered-name form-control"></th>
                                        <th><input type="text" class="elapsed form-control"></th>
                                        <th>
                                            <input type="text" class="filtered-datepicker form-control">
                                        </th>
                                    </tr>
                                    <tr>

                                    <th></th>
                                    <th></th>
                                    <th></th>
                                    <th></th>
                                    </tr>
                                </thead>
                            </table>
                          </div>
                      </div>
                    </div>
                </div>
                <div role="tabpanel" class="tab-pane" id="tab-settings">
                    <div class="panel panel-default">
                        <div class="panel-heading clearfix">
                            <h3 class="panel-title pull-left">Settings</h3>
                        </div>
                        <div class="panel-body">
                            <button type="button" class="btn btn-info dump-database"
                                    style="padding: 3px 7px; margin-bottom: 0; font-size: 13px;">Dump current database
                            </button>
                            <button type="button" class="btn btn-danger delete-database"
                                    style="padding: 3px 7px; margin-bottom: 0; font-size: 13px;">Delete all data
                            </button>

                            <div class="col-xs-12" style="margin-top: 10px;"></div>

                            <div class="col-xs-12">
                                <div class="col-xs-11"><div id="settings-info"></div></div>
                                <div class="col-xs-1">
                                    <button id="clear-settings-info" type="button" class="btn btn-default pull-right"
                                            style="padding: 3px 7px; margin-bottom: 0; font-size: 13px;">Clear
                                    </button>
                                </div>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>

</div>

<div class="modal fade filteredModal" tabindex="-1" role="dialog" aria-labelledby="filteredModal">
  <div class="modal-dialog modal-lg">
    <div class="modal-content">
    <div class="modal-body">
      </div>

    </div>
  </div>
  </div>


<script src="static/dist/js/vendor.js"></script>
<script src="static/dist/js/main.js"></script>
<script src="static/dist/js/settings.js"></script>
</body>

{% include "footer.html" %}